In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-07-11 09:42:33,403 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [2]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    # "track": "jerez gp",
    "track": "spa up",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-07-11 09:42:33,498 DEBUG Influx: Connected to https://telemetry.b4mad.racing/
2023-07-11 09:42:33,502 INFO Fetching telemetry for iRacing - spa up - Ferrari 488 GT3 Evo 2020
2023-07-11 09:42:33,502 INFO   track.id 67 car.id 9
2023-07-11 09:42:33,503 INFO   session 1683692514 lap.id 48460 number 2
2023-07-11 09:42:33,503 INFO   length 6930 time -1.0 valid True
2023-07-11 09:42:33,503 INFO   start 2023-05-10 00:26:55.185506+00:00 end 2023-05-10 00:34:58.743134+00:00
2023-07-11 09:42:34,775 INFO Fetching telemetry for iRacing - spa up - Ferrari 488 GT3 Evo 2020
2023-07-11 09:42:34,775 INFO   track.id 67 car.id 9
2023-07-11 09:42:34,775 INFO   session 1686113933 lap.id 121819 number 2
2023-07-11 09:42:34,776 INFO   length 6928 time 139.1579 valid True
2023-07-11 09:42:34,776 INFO   start 2023-06-07 07:01:17.847246+00:00 end 2023-06-07 07:03:36.962231+00:00
2023-07-11 09:42:35,407 INFO Fetching telemetry for iRacing - spa up - Ferrari 488 GT3 Evo 2020
2023-07-11 09:42:35,408 INFO   tra

In [3]:
# influx_data_frames = []
# for df in data_frames:
#     influx_data_frames.append(df.copy())

# data_frames = []
# for df in influx_data_frames:
#     data_frames.append(df.copy())

In [37]:
current_sectors = fast_lap_analyzer.current_fast_lap_sectors()
display(current_sectors)

[{'start': 22, 'end': 2162},
 {'start': 2163, 'end': 2784},
 {'start': 2785, 'end': 3088},
 {'start': 3089, 'end': 3578},
 {'start': 3579, 'end': 4235},
 {'start': 4236, 'end': 4708},
 {'start': 4709, 'end': 6927},
 {'start': 0, 'end': 21}]

In [41]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)

# are the sectors similar to the current fast lap?
similarity = fast_lap_analyzer.similar_sectors(sector_start_end, current_sectors)
display(f"Sectors are similar: {similarity}")

fig = lap_fig(df_max, full_range=True, columns=["Throttle", "Brake"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"], x1=sector["end"], color="red")
fig.show()

2023-07-11 12:04:33,686 DEBUG Lap 0 too short, skipping
2023-07-11 12:04:33,694 DEBUG remove sector 2 too close to previous sector: 27.0
2023-07-11 12:04:33,694 DEBUG remove sector 3 too close to previous sector: 39.0
2023-07-11 12:04:33,695 DEBUG remove sector 5 too close to previous sector: 97.0
2023-07-11 12:04:33,695 DEBUG remove sector 8 too close to previous sector: 37.0
2023-07-11 12:04:33,695 DEBUG remove sector 11 too close to previous sector: 10.0


[{'start': 0, 'end': 2160.0, 'length': 2160},
 {'start': 2161.0, 'end': 2784.0, 'length': 623},
 {'start': 2785.0, 'end': 3569.0, 'length': 784},
 {'start': 3570.0, 'end': 4229.0, 'length': 659},
 {'start': 4230.0, 'end': 4705.0, 'length': 475},
 {'start': 4706.0, 'end': 6433.0, 'length': 1727},
 {'start': 6434.0, 'end': 6928.74, 'length': 494}]

'Sectors are similar: False'

In [42]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
track_length = df_max["DistanceRoundTrack"].max()
for i in range(len(segments)):
    segment = segments[i]
    # display(segment.start, segment.end, segment.turn)
    # display(segment.telemetry)
    # df = telemetry_for_fig(segment)
    df = segment.telemetry
    fig = lap_fig(df, columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        # brake_features = features_for_fig(segment, track_length, brake_features)
        fig_add_features(fig, brake_features)
    throttle_features = segment.throttle_features()
    if throttle_features:
        # throttle_features = features_for_fig(segment, track_length, throttle_features)
        fig_add_features(fig, throttle_features, color="green")
    display(f"time: {segment.time} {segment.start} -> {segment.end} / track: {segment.track_length}")
    fig.show()

2023-07-11 12:04:38,793 DEBUG start: 0, end: 2160.0
2023-07-11 12:04:38,804 DEBUG start: 2161.0, end: 2784.0
2023-07-11 12:04:38,814 DEBUG start: 2785.0, end: 3569.0
2023-07-11 12:04:38,825 DEBUG start: 3570.0, end: 4229.0
2023-07-11 12:04:38,836 DEBUG start: 4230.0, end: 4705.0
2023-07-11 12:04:38,847 DEBUG start: 4706.0, end: 6433.0
2023-07-11 12:04:38,848 DEBUG sector 0: max: 5370.84 != end: 6433.0
2023-07-11 12:04:38,859 DEBUG start: 6434.0, end: 6928.74
2023-07-11 12:04:38,860 ERROR sector 0 is empty


'time: 38.7797661 0 -> 2160 / track: 6928.74'

'time: 14.199999999999996 2161 -> 2784 / track: 6928.74'

'time: 19.03333 2785 -> 3569 / track: 6928.74'

'time: 11.733329800000007 3570 -> 4229 / track: 6928.74'

'time: 10.566670000000002 4230 -> 4705 / track: 6928.74'

'time: 30.066665999999998 4706 -> 6433 / track: 6928.74'

'time: 13.75000799999998 6434 -> 6928 / track: 6928.74'